In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
from math import sqrt
from collections import Counter
from sklearn import neighbors
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")
%matplotlib inline

In [38]:
df = pd.read_csv('basketball_teams.csv')
#df

In [40]:
dfNBA = df[df.lgID == 'NBA']#Using Only NBA information for our data
dfNBA = dfNBA[dfNBA.games == 82]#Using Only NBA seasons that played the full 82 games.
dfNBA = dfNBA[df.o_fgm != 0]#Removing rows where there are no stats for certain categories.


In [41]:
df = dfNBA[df.d_fgm != 0]

In [45]:
playoff = pd.DataFrame()
df1 = playoff
df['playoff'] = df['playoff'].astype(str)

In [46]:
df['Playoff'] = df.playoff.apply(lambda x: 0 if x =='nan'  else 1 ) #Converting playoff column to binary 1 made playoff.

In [47]:
playoff['o_fga'] = df['o_fga'].tolist()
playoff['o_fgm'] = df['o_fgm'].tolist()
playoff['o_asts'] =df['o_asts'].tolist()
playoff['d_fgm'] = df['d_fgm'].tolist()
playoff['Playoff'] =df['Playoff'].tolist()
playoff['wins'] =df['won'].tolist()#Converting the columns we are going to be using to lists.
playoff['total_fg'] = playoff['o_fgm']-playoff['d_fgm'] #Creating new variable for fgs made - fgs allowed.

In [56]:
knn = neighbors.KNeighborsRegressor(n_neighbors=10)
X = playoff[['total_fg', 'o_asts','Playoff']]
Y = playoff.wins
knn.fit(X, Y) #KNN Model

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=10, p=2,
          weights='uniform')

In [57]:
score = cross_val_score(knn, X, Y, cv=5)
print(" Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2)) #KNN Model Accuracy

 Accuracy: 0.48 (+/- 0.15)


In [52]:
regr = linear_model.LinearRegression()
regr.fit(X, Y)
print('Coefficients: \n', regr.coef_)
print('Intercept: \n', regr.intercept_)

Coefficients: 
 [  3.29688324e-02   1.94911853e-03   1.29441579e+01]
Intercept: 
 29.8930254872


In [58]:
data = playoff
linear_formula = 'wins ~ total_fg+o_asts+Playoff'
lm = smf.ols(formula=linear_formula, data=data).fit()

In [59]:
lm.rsquared

0.74070221534275493

Looking at our two models it looks like our linear regression does a better job of predicting wins. While both models do an okay job, they both can be improved in many ways. (We will attempt to do so in future projects). To create both models the first feature we looked at is total field goals. We calculated this by taking the field goals made an subtracted by the field goals allowed. One would expect this to have a very high linear correlation with total wins. Because of this correlation I think it adds more value to our linear model. Next we used total offensive assists as our second feature. Again we would expect more assists leads to more field goals made (but not necessarily less field goals allowed, so the two features aren't too related) leads to more wins. Finally our categorical variable is playoffs and whether a team made it or not. If a team made it to the playoffs one would expect them to have more wins so this is an important feature. I think a linear model with our features selected and the data presented will always be more accurate than KNN. 